In [1]:
%pip install langchain


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
key = os.getenv("GOOGLE_API_KEY")

In [3]:
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI   
from langchain.tools import tool     


In [4]:
!python3 --version

Python 3.10.11


In [5]:
client = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",google_api_key=key)

In [6]:
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_swarm, create_handoff_tool

## Tools for agents


In [7]:
@tool(description="generate_case", return_direct=True)
def generate_case(name: str, date: str) -> str:
    """
    A simple tool that simulates generating an appointment booking case 
    
    Args:
        name (str): The name of the person making the appointment.
        date (str): The date of the appointment.

    Returns:
        str: Confirmation message.
    """
    return "Your case is generated successfully!"

@tool(description="book_appointment", return_direct=True)
def book_appointment(name: str, date: str) -> str:
    """
    A simple tool that simulates booking an appointment.
    
    Args:
        name (str): The name of the person making the appointment.
        date (str): The date of the appointment.
    
    Returns:
        str: Confirmation message.
    """
    return f"Appointment booked for {name} on {date} successfully!"

@tool(description="medicine_info", return_direct=True)
def medicine_info(medicine_name: str) -> str:
    """
    A simple tool that simulates retrieving information about a medicine.
    
    Args:
        medicine_name (str): The name of the medicine.
    
    Returns:
        str: Information about the medicine.
    """
    return f"Information about {medicine_name}: [Details about the medicine]"


## handoff Tools

In [8]:
transfer_to_appointment_scheduler = create_handoff_tool(
    agent_name="appointment_scheduler",
    description="Transfer user to the appointment scheduler.",
)
transfer_to_case_generator = create_handoff_tool(
    agent_name="case_generator",
    description="Transfer user to the case generator.",
)

transfer_to_medicine_inventory = create_handoff_tool(
    agent_name="medicine_inventory",
    description="Transfer user to the medicine inventory assistant.",
)
transfer_to_supervisor = create_handoff_tool(
    agent_name="supervisor",
    description="Transfer user to the supervisor.",
)

## Scheduling Agent

In [9]:
appointment_scheduler = create_react_agent(
    model=client,
    tools=[book_appointment, transfer_to_case_generator, transfer_to_medicine_inventory, transfer_to_supervisor],
    prompt="You are an appointment scheduler agent that can book appointments for users for medical consultations you can also transfer users to the other agents when your part is done.", 
    name="appointment_scheduler"
)

## Case Generator Agent

In [10]:
case_generator = create_react_agent(
    model=client,
    tools=[generate_case, transfer_to_appointment_scheduler, transfer_to_medicine_inventory, transfer_to_supervisor],
    prompt="You are a case generator that can create new cases . You will analyze the user's request and generate a structured case. You can also transfer users to the other agents when your part is done.",
    name="case_generator"
)


## Inventory Management Agent

In [11]:
medicine_inventory = create_react_agent(
    model=client,
    tools=[medicine_info, transfer_to_appointment_scheduler, transfer_to_case_generator, transfer_to_supervisor],
    prompt="You are a medicine inventory assistant that can provide information about medicines. You can also transfer users to the other agents when your part is done.",
    name="medicine_inventory"
)


## Supervisor Agent

In [12]:
supervisor = create_react_agent(
    model=client,
    tools=[transfer_to_appointment_scheduler, transfer_to_case_generator, transfer_to_medicine_inventory],
    prompt="You are a supervisor that can make decisions about which assistant to transfer the user to fullfilling their request. you break the the request down into smaller tasks and decide which assistant is best suited to handle each task." ,
    name="supervisor"
)

## ToolMessage Handler

In [13]:
from langchain_core.messages import ToolMessage, AIMessage
from typing import Dict, Any

def inject_tool_messages(state: Dict[str, Any]) -> Dict[str, Any]:
    for agent_name, agent_state in state.items():
        if not isinstance(agent_state, dict):
            continue
        messages = agent_state.get("messages", [])
        if not messages:
            continue

        for msg in reversed(messages):
            if isinstance(msg, AIMessage) and msg.tool_calls:
                for tool_call in msg.tool_calls:
                    # Only inject if not already injected
                    if not any(
                        isinstance(m, ToolMessage) and m.tool_call_id == tool_call["id"]
                        for m in messages
                    ):
                        messages.append(
                            ToolMessage(
                                content=f"Simulated response from tool `{tool_call['name']}`.",
                                name=tool_call["name"],
                                tool_call_id=tool_call["id"]
                            )
                        )
                break  # Only handle the most recent tool call

        agent_state["messages"] = messages
    return state


## Create checkpointer

In [14]:
from langgraph.checkpoint.memory import InMemorySaver

memory = InMemorySaver()


## Create Swarm

In [15]:
swarm = create_swarm(
    agents=[appointment_scheduler, case_generator, medicine_inventory, supervisor],
    default_active_agent="supervisor",
).compile(checkpointer=memory)

## configuration to pass while streaming

In [16]:
config = {"configurable": {"thread_id": "1"}}

In [17]:
from pprint import pprint

In [18]:
graph_state = swarm.get_state(config)
print("Graph state:", graph_state)

Graph state: StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())


In [19]:
state = {
   
        "messages": [
            {"role": "user", "content": "I need to book an appointment for John Doe on 2023-10-15 and check Aspirin availability."}
        ]
    
}



In [22]:
# Step 1: Get state and access internal dict
graph_state = swarm.get_state(config)
graph_state_dict = graph_state.values 
print("-----------------------------------------------")
print(graph_state_dict) # ✅ Correct way
print("-----------------------------------------------")

# Step 2: Patch tool messages
def patch_tool_messages(state):
    if state:
        messages = state["messages"]
        patched_messages = list(messages)
    

        for msg in reversed(messages):
            if isinstance(msg, AIMessage) and msg.tool_calls:
                for tool_call in msg.tool_calls:
                    call_id = tool_call["id"]

                    already_responded = any(
                        isinstance(m, ToolMessage) and m.tool_call_id == call_id
                        for m in messages
                    )

                    if not already_responded:
                        print(f"Injecting ToolMessage for tool_call: {tool_call}")
                        patched_messages.append(
                            ToolMessage(
                                content=f"Simulated response from {tool_call['name']}.",
                                tool_call_id=call_id,
                                name=tool_call["name"]
                            )
                        )
                break

        return {"messages": patched_messages}
    else:
        return {"messages":[]}
# Step 3: Update & resume
patched_state = patch_tool_messages(graph_state_dict)
swarm.update_state(config, patched_state)

# Step 4: Resume execution
res = swarm.invoke( state, config=config)
pprint(res)


-----------------------------------------------
{}
-----------------------------------------------


exception calling callback for <Future at 0x7fd14c687df0 state=finished raised ParentCommand>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/concurrent/futures/_base.py", line 342, in _invoke_callbacks
    callback(self)
  File "/home/tejas.raval@simform.dom/Desktop/ViniGraph/pheonix/lib/python3.10/site-packages/langgraph/pregel/runner.py", line 107, in on_done
    self.callback()(task, _exception(fut))  # type: ignore[misc]
  File "/home/tejas.raval@simform.dom/Desktop/ViniGraph/pheonix/lib/python3.10/site-packages/langgraph/pregel/runner.py", line 434, in commit
    raise exception
  File "/home/tejas.raval@simform.dom/Desktop/ViniGraph/pheonix/lib/python3.10/site-packages/langgraph/pregel/executor.py", line 80, in done
    task.result()
  File "/usr/local/lib/python3.10/concurrent/futures/_base.py", line 451, in result
    return self.__get_result()
  File "/usr/local/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
    raise self._excepti

ValueError: Found AIMessages with tool_calls that do not have a corresponding ToolMessage. Here are the first few of those tool calls: [{'name': 'book_appointment', 'args': {'date': '2023-10-15', 'name': 'John Doe'}, 'id': '74b5407e-e59f-49e6-9bdc-f58739e8f6f4', 'type': 'tool_call'}].

Every tool call (LLM requesting to call a tool) in the message history MUST have a corresponding ToolMessage (result of a tool invocation to return to the LLM) - this is required by most LLM providers.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_CHAT_HISTORY

("appointment_scheduler : content='I need to book an appointment for a patient "
 'named John Doe on 2023-10-15. Also, I need to check the availability of '
 "Aspirin in the medicine inventory.' additional_kwargs={} "
 "response_metadata={} id='fadf7364-213c-4176-94a2-098cb3556e4e'")


("appointment_scheduler : content='I can help you with that. First, I will "
 'transfer you to the appointment scheduler to book the appointment for John '
 'Doe. After that, I will transfer you to the medicine inventory assistant to '
 "check the availability of Aspirin.' additional_kwargs={'function_call': "
 "{'name': 'transfer_to_appointment_scheduler', 'arguments': '{}'}} "
 "response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': "
 "[]}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', "
 "'safety_ratings': []} name='supervisor' "
 "id='run--48035f99-e1f3-44ce-bd20-376e92818436-0' tool_calls=[{'name': "
 "'transfer_to_appointment_scheduler', 'args': {}, 'id': "
 "'dab

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm
from dotenv import load_dotenv
load_dotenv()


def add(a: int, b: int) -> int:
    '''Add two numbers'''
    return a + b

alice = create_react_agent(
    client,
    [add, create_handoff_tool(agent_name="Bob")],
    prompt="You are Alice, an addition expert.",
    name="Alice",
)

bob = create_react_agent(
    client,
    [create_handoff_tool(agent_name="Alice", description="Transfer to Alice, she can help with math")],
    prompt="You are Bob, you speak like a pirate.",
    name="Bob",
)

checkpointer = InMemorySaver()
workflow = create_swarm(
    [alice, bob],
    default_active_agent="Alice"
)
app = workflow.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "1"}}
turn_1 = app.invoke(
    {"messages": [{"role": "user", "content": "i'd like to speak to Bob"}]},
    config,
)
turn_2 = app.invoke(
    {"messages": [{"role": "user", "content": "what's 5 + 7?"}]},
    config,
)

In [ ]:
from pprint import pprint
pprint(turn_1)
print("-----")
pprint(turn_2)

{'active_agent': 'Bob',
 'messages': [HumanMessage(content="i'd like to speak to Bob", additional_kwargs={}, response_metadata={}, id='b17c4a9f-f3a2-429d-889b-bf621276b643'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'transfer_to_bob', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, name='Alice', id='run--c4dc20cc-4147-4f72-925c-14d4c5567fb0-0', tool_calls=[{'name': 'transfer_to_bob', 'args': {}, 'id': '1769640b-9350-4e8c-9da8-ef0b8ec528de', 'type': 'tool_call'}], usage_metadata={'input_tokens': 39, 'output_tokens': 5, 'total_tokens': 44, 'input_token_details': {'cache_read': 0}}),
              ToolMessage(content='Successfully transferred to Bob', name='transfer_to_bob', id='5164b63d-796f-480f-b25e-4276eb4ee833', tool_call_id='1769640b-9350-4e8c-9da8-ef0b8ec528de'),
              AIMessage(content='Aye matey! I 